# **SPAM vs. naiwny klasyfikator Bayesa**

## Wprowadzenie
Nigeryjski książę wciąż zarabia na użytkownikach elektronicznych skrzynek pocztowych ponad 700 tys. dolarów rocznie ([źródło](https://www.cnbc.com/2019/04/18/nigerian-prince-scams-still-rake-in-over-700000-dollars-a-year.html))! Jak to możliwe?

Pierwsza przyczyna jest natury psychologicznej. Ofiary są poddawane "perfekcyjnej burzy pokuszeń", jak ujął to psycholog w wywiadzie, do którego linka dałam Wam powyżej. Spammerzy łączą granie na ludzkiej chciwości, ale także na pragnieniu bycia bohaterem. W końcu kto nie chciałby zarobić na byciu wspaniałomyślnym i szczodrym? W tej kwestii możemy pracować wyłącznie nad sobą.

Możemy za to pracować nad filtrami antyspamowymi. Użyjemy techniki, która nazywa się "worek ze słowami" (bag of words) w połączeniu z naiwnym klasyfikatorem Bayesa. Choć to prosty klasyfikator, z powodzeniem jest używany współcześnie (np. [SpamAssassin](https://cwiki.apache.org/confluence/display/spamassassin/BayesInSpamAssassin)).

Notebook oparty na tutorialach:
*   https://towardsdatascience.com/spam-classifier-in-python-from-scratch-27a98ddd8e73
*   https://towardsdatascience.com/spam-filtering-using-naive-bayes-98a341224038


## Import danych treningowych
https://www.kaggle.com/uciml/sms-spam-collection-dataset

To dane przygotowane przez Almeida et al. na podstawie forum brytyjskiego, gdzie użytkownicy skarżą się na spamowe SMSy. Każdy wiersz składa się z kolumny opisującej czy wiadomość jest spamem, czy nie ('spam' czy 'ham'), a druga zawiera treść wiadomości.

Jak na ćwiczeniach o regresji logistycznej, możemy pobrać zestaw danych z GitHuba:

In [ ]:
!git clone https://github.com/Shmoo137/uczenie-maszynowe-2021-22

In [ ]:
folder = '/content/uczenie-maszynowe-2021-22/dane/' # tu się domyślnie zapisuje pobrane repo z GitHuba, a dane są w folderze "dane"

Do pracy z danymi tekstowymi bardzo przydatna jest biblioteka [pandas](https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/).

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
mails = pd.read_csv(folder + 'spam_dataset.csv', encoding='latin-1')
mails.head()

Wyczyśćmy ten zbiór danych. Usuńmy niepotrzebne kolumny i zastąpmy nazwy 'v1' i 'v2' czymś bardziej przyjaznym.

In [ ]:
mails = mails.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
mails.head()

In [ ]:
mails = mails.rename(columns={"v1": "klasa", "v2": "tekst"})
mails.head()

Zobaczmy jak wyglądają przykładowe dane o numerze jakimkolwiek.

In [ ]:
id = 57

Treść wiadomości:

In [ ]:
mails['tekst'][id]

Kategoria:

In [ ]:
mails['klasa'][id]

Ile mamy tych maili?

In [ ]:
mails.shape

## Analiza częstości występowania słów w obu klasach za pomocą biblioteki WordCloud

To biblioteka pozwalająca generować śliczne obrazki, na których wielkość słów odpowiada częstości jego występowania w danym zbiorze.

In [ ]:
!pip install wordcloud
import wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

spam_words = " ".join(list(mails [mails['klasa']=='spam']['tekst'] ))
spam_plot = WordCloud(width = 512, height = 512).generate(spam_words)

plt.figure(figsize=(10,8))
plt.imshow(spam_plot)

Zrób to samo dla smsów nie będących spamem:

In [ ]:
ham_words = ...

Przygotujmy dane do treningu i testu klasyfikatora:

In [ ]:
from sklearn.model_selection import train_test_split

X = mails.tekst
y = mails.klasa
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

Przekodowujemy wiadomości na wektory cech.  Korzystamy z funkcji: [sklearn.feature_extraction.text.CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = ... # stwórz instancje obiektu CountVectorizer
X_train = ... # naucz vectorizer słownika i przetransformuj dane uczące

Wypisz rozmiary danych treningowych:

In [ ]:
print("Dane treningowe: n_samples: %d, n_features: %d" % X_train.shape)

Dane uczące są przechowywane w macierzy rzadkiej (sparse matrix). Proszę podejrzeć jak wyglądają tak przekodowane dane:

In [ ]:
print(X_train[57])

 Wektoryzujemy też dane testowe, wykorzystując już stworzony na podstawie danych treningowych wektor słów:

In [ ]:
X_test = ... # nie twórz nowego vectorizera, użyj gotowego, inaczej maszyna nie będzie potrafiła analizować danych testowych
print("Dane testowe: n_samples: %d, n_features: %d" % X_test.shape)

Odwrotne mapowanie z cech na słowa:

In [ ]:
feature_names = vectorizer.get_feature_names()
feature_names = np.asarray(feature_names)

Tworzymy instancję i uczymy klasyfikator MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = ...
clf.fit...

## Ocena jakości: jak zwykle będziemy korzystać z funkcji zaimplementowanych w [sci-kit](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)

In [ ]:
y_pred = ... # obliczamy predykcję dla tekstów ze zbioru testowego

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accur = ... # dokładność
print("Dokładność: %0.3f" % accur)
print("Classification report:") # wypisz raport klasyfikacji 
...

print("Macierz błędów") # wypisz macierz (confusion matrix)
...

Sprawdźmy, czego właściwie maszyna się nauczyła:

In [ ]:
print("Słowa, które z największą pewnością wskazują maszynie, że wiadomość to spam:")
top10 = np.argsort(clf.coef_[0])[-10:]
bottom10 = np.argsort(clf.coef_[0])[:10]
print(feature_names[top10])

print("Słowa najmniej istotne przy klasyfikacji:")
print(feature_names[bottom10])

## Zastanówmy się, czy możemy jakoś ułatwić zadanie maszynie, wykorzystując naszą znajomość języka

### Wygładzanie Laplace'a

Poszukajcie słowa, które nie występuje w zbiorze treningowym.

In [ ]:
id = np.where(feature_names == ...)
print(id)

our_message = ...
vectorized_message = vectorizer.transform([our_message])
clf.predict(vectorized_message)

### Stemming (nawet nie będę próbować tego tłumaczyć na polski, to [bogate](https://pl.bab.la/slownik/angielski-polski/stemming) znaczeniowo słowo)

Polega na ujednoliceniu słów o tym samym rdzeniu znaczeniowym (o czym maszyna, oczywiście, nie ma szans wiedzieć). Np. dzięki stemmingowi słowa "go", "going" i "goes" są przyporządkowane tylko jednemu słowu "go". Można np. użyć gotowego algorytmu stemmingowego o nazwie [Porter Stemmer](https://tartarus.org/martin/PorterStemmer/).

In [ ]:
id1 = np.where(feature_names == 'going')
id2 = np.where(feature_names == 'go')
print(id1, id2)

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

message = 'Applying classical methods of machine learning to the study of quantum systems (sometimes called quantum machine learning) is the focus of an emergent area of physics research'
words = word_tokenize(message)

stemmer = PorterStemmer()
words = [stemmer.stem(word) for word in words]

print(words)

Powtórzmy trening i testowanie naszego klasyfikatora na danych poddanych stemmingowi:

In [ ]:
stemmer = PorterStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

vectorizer = CountVectorizer(analyzer=stemmed_words)

In [ ]:
X = ...
y = ...
X_train, X_test, y_train, y_test = ...
# vectorizer z PorterStemmerem został już stworzony w okienku wyżej
X_train = vectorizer.fit_transform(...) # naucz vectorizer słownika i przetransformuj dane uczące
X_test = ... # przetransformuj dane uczące korzystając z tego samego słownika
clf = ...
clf.fit...

y_pred = ...
accur = ... # dokładność
print("Dokładność: %0.3f" % accur)
print("Classification report:") # wypisz raport klasyfikacji 
...

print("Macierz błędów") # wypisz macierz (confusion matrix)
...

feature_names = vectorizer.get_feature_names()
feature_names = np.asarray(feature_names)

print("Słowa, które z największą pewnością wskazują maszynie, że wiadomość to spam:")
top10 = np.argsort(clf.coef_[0])[-10:]
bottom10 = np.argsort(clf.coef_[0])[:10]
print(feature_names[top10])

print("Słowa najmniej istotne przy klasyfikacji:")
print(feature_names[bottom10])

## Gdybyście byli spammerami... Co moglibyście zrobić, znając tę technikę antyspamową?

### Stosowanie znaków specjalnych zamiast liter

In [ ]:
our_message = vectorizer.transform(['call for free'])
print(clf.predict(our_message))

our_tricky_message = vectorizer.transform(['c@ll for free'])
print(clf.predict(our_tricky_message))

### Wysyłanie obrazków z tekstem!
-> nakładki OCR (ang. optical character recognition)

Jakieś inne pomysły? :)